In [1]:
#importamos los paquetes necesarios
import cv2
import PIL
import numpy as np
import os

from PIL import Image

In [2]:
#definimos la función encargada de recortar la imagen y ajustarla a los bordes
def crop_image_from_gray(img, tol=7):
    '''
    Recorta una imagen (de fondo de ojo) para ajustarla a los bordesy eliminar el espacio de fondo sobrante. Además transforma la imagen a escala de grises.
    
    Parámetros
    -----------------------------------
    img: se trata de la imagen sobre la cual se desea aplicar la transformación. Puede ser un array de Numpy.
    tol: número entero que representa el valor mínimo que ha de tener un píxel de la imagen para no ser considerado como fondo a la hora de realizar la máscara.
    
    Return
    -----------------------------------
    Devuelve en todos los casos la imagen pasada como parámetro pero ya recortada y procesada (blanco y negro)
    '''
    #Verifica la dimensionalidad de la imagen de entrada, si es dos (blanco y negro):
    if img.ndim == 2:
        #Crea una máscara con los píxeles de la imagen que superan la tolerancia pasada como parámetro (para eliminar el fondo)
        mask = img > tol
        #Recorta la imagen utilizando la máscara y la devuelve
        return img[np.ix_(mask.any(1), mask.any(0))]
    #Si la imagen tiene tres dimensiones (RGB)
    elif img.ndim == 3:
        #Convierte la imagen a escala de grises utilizando cv2
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        #Crea una máscara con los píxeles de la imagen en escala de grises que superan la tolerancia (para eliminar el fondo)
        mask = gray_img > tol
        
        #Verifica si la forma de la imagen recortada es válida
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        #Si la forma no es válida, devuelve la imagen sin recortar
        if (check_shape == 0):
            return img
        #Si la forma es válida
        else:
            #Recorta cada canal de la imagen utilizando la máscara
            img1 = img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2 = img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3 = img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            # Une los canales recortados en una sola imagen
            img = np.stack([img1,img2,img3],axis=-1)
        # Devuelve la imagen recortada
        return img

In [3]:
def circle_crop(img, sigmaX):   
    """
    Create circular crop around image centre    
    """    
    img = crop_image_from_gray(img)    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img=cv2.addWeighted(img,4, cv2.GaussianBlur( img , (0,0) , sigmaX) ,-4 ,128)
    return img 